# Qlib 机器学习模型训练示例

这个 Notebook 展示了如何使用 AutoTrade 内部组件进行机器学习模型的完整训练流程，包括：
1. **数据加载**：使用 `QlibDataAdapter` 获取股票历史数据。
2. **特征工程**：使用 `QlibFeatureGenerator` 自动生成 Alpha 特征。
3. **目标准备**：计算未来收益率作为训练目标。
4. **模型训练**：使用 `LightGBMTrainer` 训练回归模型。
5. **模型评估**：计算 IC, ICIR 等指标。
6. **保存与管理**：使用 `ModelManager` 管理模型版本。

In [ ]:
import os
import sys
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import numpy as np

# 确保可以导入项目模块
root_dir = Path(os.getcwd()).parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

from autotrade.research.data import QlibDataAdapter
from autotrade.research.features import QlibFeatureGenerator
from autotrade.research.models import LightGBMTrainer, ModelManager

print(f"Project root: {root_dir}")

## 1. 数据准备

我们选择一组常见的股票标的进行训练。

In [ ]:
symbols = ["SPY", "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA"]

# 初始化数据适配器
adapter = QlibDataAdapter(data_dir=root_dir / "data/qlib")

# 获取数据（如果本地没有则会通过 Yahoo Finance 下载）
end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 3) # 近3年数据

print(f"Fetching and loading data for {len(symbols)} symbols from {start_date.date()} to {end_date.date()}...")
df = adapter.load_data(symbols, start_date=start_date, end_date=end_date)

print(f"Total rows: {len(df)}")
df.head()

## 2. 特征工程

使用项目内置的 `QlibFeatureGenerator` 生成标准化特征。

In [ ]:
feature_gen = QlibFeatureGenerator(normalize=True)
features = feature_gen.generate(df)

print(f"Features generated: {features.shape[1]} descriptors")
features.head()

## 3. 目标变量 (Labels)

我们要预测未来 $N$ 天的累计收益率。通常 $N=5$ (一周) 是一个不错的中短期目标。

In [ ]:
target_horizon = 5

# 计算未来收益率
close_prices = df["close"].unstack("symbol")
future_returns = close_prices.pct_change(target_horizon).shift(-target_horizon)
target = future_returns.stack().reindex(features.index)

# 清洗数据：移除包含 NaN 的行（特征或目标缺失）
valid_mask = ~(features.isna().any(axis=1) | target.isna())
features = features[valid_mask]
target = target[valid_mask]

print(f"Cleaned dataset size: {len(features)}")

## 4. 模型训练

我们将数据集分为训练集和验证集（按时间切分，模拟真实部署场景）。

In [ ]:
# 时间切分点：前 80% 训练，后 20% 验证
split_date = features.index.get_level_values(0).unique().sort_values()[int(len(features.index.levels[0]) * 0.8)]
print(f"Splitting data at {split_date.date()}")

X_train = features[features.index.get_level_values(0) < split_date]
X_valid = features[features.index.get_level_values(0) >= split_date]
y_train = target[target.index.get_level_values(0) < split_date]
y_valid = target[target.index.get_level_values(0) >= split_date]

# 初始化并训练 LightGBM
trainer = LightGBMTrainer(
    model_dir=root_dir / "models",
    model_name="example_notebook_model",
    num_boost_round=300
)

print("Starting training...")
trainer.train(X_train, y_train, X_valid, y_valid)
print("Training finished.")

## 5. 评估

计算因子投资中常用的 IC (Information Coefficient) 指标。

In [ ]:
metrics = trainer.evaluate(X_valid, y_valid)

print("\n--- Validation Metrics ---")
print(f"IC: {metrics['ic']:.4f}")
print(f"ICIR: {metrics['icir']:.4f}")
print(f"MSE: {metrics['mse']:.6f}")

# 特征重要性
print("\nTop 10 Features:")
importance = trainer.get_feature_importance(10)
display(importance)

## 6. 保存并启用模型

保存训练好的模型，并可以在 Web UI 的“模型管理”中看到它。

In [ ]:
# 更新元数据
trainer.metadata.update({
    "symbols": symbols,
    "ic": metrics["ic"],
    "icir": metrics["icir"],
    "target_horizon": target_horizon,
    "updated_at": datetime.now().isoformat()
})

# 保存文件
model_path = trainer.save()
print(f"Model saved to: {model_path}")

# (可选) 设为当前生产模型
# manager = ModelManager(root_dir / "models")
# manager.set_current_model(model_path.name)
# print(f"Model '{model_path.name}' is now active for live trading!")